# Inspection Data Prep

This notebook pulls a snapshot of data from the [Durham Open Data](https://opendurham.nc.gov/explore/?sort=modified) site. It preps a copy of the data for use in a hands-on tutorial session where everyone hitting the portal would be A Bad Thing.

In [49]:
import os
import pandas as pd
from collections import OrderedDict
from datetime import datetime

In [50]:
pd.options.display.max_columns = 200

Restaurants data from https://opendurham.nc.gov/explore/dataset/restaurants-data/. Really, it's establishments since there's far more than restaurants in here.

In [51]:
if not os.path.isfile('establishments_20160617_raw.csv'):
    !wget -O establishments_20160617_raw.csv 'https://opendurham.nc.gov/explore/dataset/establishments-data/download/?format=csv&timezone=America/New_York&use_labels_for_header=true'
establishments = pd.read_csv('./establishments_20160617_raw.csv', sep=';')

Health inspection data from https://opendurham.nc.gov/explore/dataset/inspection-data/.

In [52]:
if not os.path.isfile('inspections_20160617_raw.csv'):
    !wget -O inspections_20160617_raw.csv 'https://opendurham.nc.gov/explore/dataset/inspection-data/download/?format=csv&timezone=America/New_York&use_labels_for_header=true'
inspections = pd.read_csv('./inspections_20160617_raw.csv', sep=';')

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Inspection violation data from https://opendurham.nc.gov/explore/dataset/violation-data/.

In [53]:
if not os.path.isfile('violations_20160617_raw.csv'):
    !wget -O violations_20160617_raw.csv 'https://opendurham.nc.gov/explore/dataset/violation-data/download/?format=csv&timezone=America/New_York&use_labels_for_header=true'
violations = pd.read_csv('./violations_20160617_raw.csv', sep=';')

## Establishments 

Let's only deal with food service and mobile food establishments.

In [54]:
establishments.Rpt_Area_Desc.value_counts()

Food Service             1075
Swimming Pools            416
Summer Food               221
Day Care                  173
Residential Care          144
Mobile Food               140
School Buildings           88
Lodging                    60
Tattoo Establishments      36
Institutions               30
Bed&Breakfast Home          4
Adult Day Care              3
Local Confinement           2
Summer Camps                2
Bed&Breakfast Inn           2
Name: Rpt_Area_Desc, dtype: int64

In [55]:
establishments = establishments[(establishments.Rpt_Area_Desc == 'Food Service') | (establishments.Rpt_Area_Desc == 'Mobile Food')]

In [56]:
len(establishments)

1215

In [57]:
establishments.head(1)

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,LatLon
1,166934,MCDONALDS 27133,5507 S MIAMI BLVD,NaN,DURHAM,NC,27703,(919) 677-2700,NaN,2015-07-13,NaN,90,5 - Municipal/Community,4 - On-Site System,2,NaN,2,NaN,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.8771424, -78.8509667"


In [58]:
to_keep = OrderedDict()
to_keep['ID'] = 'id'
to_keep['Premise_Name'] = 'name'
to_keep['Opening_Date'] = 'opened'
to_keep['Closing_Date'] = 'closed'
to_keep['Insp_Freq'] = 'insp_freq'
to_keep['Type_Description'] = 'type'
to_keep['LatLon'] = 'latlon'

In [59]:
establishments_clean = establishments[list(to_keep.keys())].rename(columns=to_keep).reset_index(drop=True)

In [60]:
establishments_clean.head(1)

,id,name,opened,closed,insp_freq,type,latlon
0,166934,MCDONALDS 27133,2015-07-13,NaN,2,1 - Restaurant,"35.8771424, -78.8509667"


In [61]:
establishments_clean.dtypes

id            int64
name         object
opened       object
closed       object
insp_freq     int64
type         object
latlon       object
dtype: object

In [62]:
establishments_clean['opened'] = pd.to_datetime(establishments_clean.opened)
establishments_clean['closed'] = pd.to_datetime(establishments_clean.closed)

In [63]:
establishments_clean.dtypes

id                    int64
name                 object
opened       datetime64[ns]
closed       datetime64[ns]
insp_freq             int64
type                 object
latlon               object
dtype: object

## Inspections

Keep inspections for these establishments only.

In [64]:
inspections = inspections[inspections.Est_Id.isin(establishments_clean.id)]

In [65]:
len(inspections)

34449

In [66]:
inspections.head(1)

,Id,Premise_Name,Premise_City,District,County,Est_Id,State_Id,Request_Number,Ehs_Id,Ehs,Territory,Est_Type,Insp_Type,Insp_Date,Extra_Credit,Grade,Score_SUM,Final_Score_SUM,Seats,Inspection_Time_Hrs,Inspection_Time_Min,Sample,Setup_Date,Update_Date,Update_User_Id,Water,Sewage,Origin,Permit_Status_Id,Permit_Status,Time_Of_Inspection_Hr,Time_Of_Inspection_Mm,Ampm_Of_Inspection,Comments,Six_Point_Demerit,Oss_Id,Rpt_Area_Code,Rpt_Area_Desc,Group_Code_Id,Time_Of_Inspection,Permit_Type_Id,Complaint_Section,Epi_Type_Id,Epi_Type_Desc,Followup_Id,Travel_Time_Hrs,Travel_Time_Min,Mileage,Sample_Attendance,Void_Date,Delete_Mark,Est_Num,Com_Num,Oss_Num,Ehs_Num,Rpt_Area_Num,Est_Type_Num,Cdp_Est_Num,Violations_Id,Field59,Comment_Sheet_Id,Action_Code_Id,Action_Code_Desc,Image_File_Path,Image_File_Name,Chlorine_SUM,Bromine_SUM,Biguanide_SUM,Water_Ph_SUM,Water_Temp_SUM,Smoking_Allowed,Classification_Id,Classification_Desc,End_Time_Of_Inspection_Hh,End_Time_Of_Inspection_Mm,End_Inspection_Ampm,Sent_To_Bets_Yn,Verification_Required_Date,Inspection_Reason_Id,Inspection_Reason_Desc,Num_Rf_Inter_Viol,Num_Repeat_Rf_Inter_Viol,Est_Group_Id,Est_Group_Desc,Person_Incharge_Lname,Person_Incharge_Fname,Type_description,LatLon
106,911494,CAPTAIN D'S,DURHAM,0,32,56936,4032011570,NaN,484,9999,NaN,1,1,2005-07-27,N,A,100,100,NaN,0,0,N,2008-05-22,2014-02-28,NaN,1,1,5,1,NaN,NaN,NaN,NaN,NaN,N,NaN,605,Food Service,1,NaN,NaN,NaN,NaN,NaN,NO,0,0,NaN,NaN,NaN,ACTIVE,4032011570,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 - INSPECTION,"36.0552119, -78.9034271"


In [67]:
to_keep = OrderedDict()
to_keep['Id'] = 'id'
to_keep['Est_Id'] = 'est_id'
to_keep['Insp_Date'] = 'date'
to_keep['Extra_Credit'] = 'extra_credit'
to_keep['Grade'] = 'grade'
to_keep['Score_SUM'] = 'score'
to_keep['Final_Score_SUM'] = 'final_score'
to_keep['Type_description'] = 'type'

In [68]:
inspections_clean = inspections[list(to_keep.keys())].rename(columns=to_keep).reset_index(drop=True)

In [69]:
inspections_clean.head(1)

,id,est_id,date,extra_credit,grade,score,final_score,type
0,911494,56936,2005-07-27,N,A,100,100,1 - INSPECTION


In [70]:
inspections_clean.dtypes

id                int64
est_id            int64
date             object
extra_credit     object
grade            object
score           float64
final_score     float64
type             object
dtype: object

In [71]:
inspections_clean['date'] = pd.to_datetime(inspections_clean.date)

In [72]:
inspections_clean.dtypes

id                       int64
est_id                   int64
date            datetime64[ns]
extra_credit            object
grade                   object
score                  float64
final_score            float64
type                    object
dtype: object

In [73]:
inspections_clean[inspections_clean['date'] < datetime(2000, 1, 1)].head(50)

,id,est_id,date,extra_credit,grade,score,final_score,type
122,881675,55474,1991-07-02,N,NaN,0,0,P - PERMIT
123,881678,55474,1992-02-03,N,NaN,0,0,1 - INSPECTION
124,881679,55474,1992-04-08,N,NaN,0,0,1 - INSPECTION
125,881680,55474,1992-07-07,N,NaN,0,0,1 - INSPECTION
126,881682,55474,1993-02-03,N,NaN,0,0,1 - INSPECTION
127,881696,55474,1996-10-14,N,NaN,0,0,1 - INSPECTION
128,881704,55474,1999-08-04,Y,A,100,102,1 - INSPECTION
133,881744,55475,1993-03-17,N,NaN,0,0,1 - INSPECTION
134,881759,55475,1996-04-10,N,NaN,0,0,1 - INSPECTION
135,881761,55475,1996-10-04,N,NaN,0,0,1 - INSPECTION


The inspection data from before year 2000 is primarily zeros. Let's start in Y2K.

In [74]:
inspections_clean = inspections_clean[inspections_clean.date > datetime(2000, 1, 1)]

## Violations

Keep violations for these inspections alone.

In [75]:
len(violations)

152098

In [76]:
violations = violations[violations.Inspection_Id.isin(inspections_clean.id)]

In [77]:
len(violations)

101299

In [78]:
violations.head(1)

,Id,Inspection_Id,Item,Weight_SUM,Critical,Comments,Rpt_Area_Desc
2,730091,946069,14,1,N,White plastic grocery bags not food grade and ...,NaN


In [79]:
to_keep = OrderedDict()
to_keep['Id'] = 'id'
to_keep['Inspection_Id'] = 'insp_id'
to_keep['Weight_SUM'] = 'weight'
to_keep['Critical'] = 'critical'
to_keep['Comments'] = 'comments'

In [80]:
violations_clean = violations[list(to_keep.keys())].rename(columns=to_keep).reset_index(drop=True)

In [81]:
len(violations_clean)

101299

In [82]:
violations_clean.head(1)

,id,insp_id,weight,critical,comments
0,730091,946069,1,N,White plastic grocery bags not food grade and ...


In [83]:
violations_clean.dtypes

id            int64
insp_id       int64
weight      float64
critical     object
comments     object
dtype: object

In [84]:
violations_clean.critical.value_counts(dropna=False)

NaN    86116
N       9385
Y       5796
0          2
Name: critical, dtype: int64

## Example Joins

We'll persist the clean copies separately. But let's make sure they can join cleanly.

In [85]:
restaurant_inspections = pd.merge(establishments_clean, inspections_clean, left_on='id', right_on='est_id', suffixes=('_est', '_insp'))

In [86]:
restaurant_inspection_violations = pd.merge(restaurant_inspections, violations_clean, left_on='id_insp', right_on='id', suffixes=('', '_viol')) 

In [87]:
restaurant_inspection_violations.head(5)

,id_est,name,opened,closed,insp_freq,type_est,latlon,id_insp,est_id,date,extra_credit,grade,score,final_score,type_insp,id,insp_id,weight,critical,comments
0,160947,PENN STATION EAST COAST SUBS,2014-10-20,NaT,2,1 - Restaurant,"35.9041882, -78.9534773",2483026,160947,2014-11-17,N,A,98,98,1 - INSPECTION,2483026,1572265,0,NaN,NaN
1,161058,FRESH HEALTHY CAFE,2014-10-27,NaT,2,2 - Food Stands,"35.9043093, -78.9411284",2489465,161058,2014-12-15,N,NaN,NaN,NaN,VERIFICATION,2489465,1573494,0,NaN,"keep food above sanitizer, store raw unpasteur..."
2,58060,BETHESDA SCHOOL LUNCHROOM,1991-09-19,NaT,4,11 - Public School Lunch,"35.9421575, -78.8386141",2484784,58060,2014-11-24,N,A,97,97,1 - INSPECTION,2484784,1572530,0,NaN,NaN
3,58060,BETHESDA SCHOOL LUNCHROOM,1991-09-19,NaT,4,11 - Public School Lunch,"35.9421575, -78.8386141",2297505,58060,2013-10-09,N,A,97,97,1 - INSPECTION,2297505,1527559,0,NaN,NaN
4,57804,TAQUERIA LA ESPERANZA,2007-01-08,NaT,4,2 - Food Stands,"35.9944791, -78.8828683",1320205,57804,2011-01-25,N,NaN,NaN,NaN,V - VISIT,1320205,1172326,NaN,Y,NaN


In [88]:
restaurant_inspection_violations.dtypes

id_est                   int64
name                    object
opened          datetime64[ns]
closed          datetime64[ns]
insp_freq                int64
type_est                object
latlon                  object
id_insp                  int64
est_id                   int64
date            datetime64[ns]
extra_credit            object
grade                   object
score                  float64
final_score            float64
type_insp               object
id                       int64
insp_id                  int64
weight                 float64
critical                object
comments                object
dtype: object

## Persist

Save 'em.

In [89]:
establishments_clean.to_pickle('establishments_20160617.pickle')

In [90]:
establishments_clean.to_csv('establishments_20160617.csv', index=False)

In [91]:
inspections_clean.to_pickle('inspections_20160617.pickle')

In [92]:
inspections_clean.to_csv('inspections_20160617.csv', index=False)

In [93]:
violations_clean.to_pickle('violations_20160617.pickle')

In [94]:
violations_clean.to_csv('violations_20160617.csv', index=False)